# HTTP C2 Beaconing Detection using Statistical Analysis

# Statistics 101
## Median:
The median of a finite list of numbers is the "middle" number, when those numbers are listed in order from smallest to greatest. 
For example, the median value of the data (1, 1, 2, 2, 4, 6, 9) is 2 
## Median Absolute Deviation:  
MAD is a measurement of how wide or narrow the distribution of the values is.
Consider the data (1, 1, 2, 2, 4, 6, 9). It has a median value of 2. The absolute deviations about 2 are (1, 1, 0, 0, 2, 4, 7) which in turn have a median value of 1 (because the sorted absolute deviations are (0, 0, 1, 1, 2, 4, 7)). So the median absolute deviation for this data is 1.  

## Skewness 
Skewness is a measure of the asymmetry of the distribution of a real-valued random variable about its mean(average)  

![skewness](./images/skewness.png)

# Regular user traffic vs. Beacon traffic

Sample beacon config:  

|Sleep|Jitter|Sleep interval|CS Sleep interval
|--|--|--|--|
|300 seconds| 20% | 240-360 seconds|240-300 seconds|

~300 connections in 24 hours.  
*Sleep = time delta  

### Users randomly visit/refresh the same web page  

---

### Beacon traffic: 300 time deltas, each between 240-360 seconds


### User traffic: 300 time deltas, each between 60-600 seconds (assumption)

![Normal vs Beacon](./images/Normal-vs-Beacon.png)

- Median of time delta of beacon traffic: 319 
- Median of time delta of user traffic: 236
- MAD of time delta of beacon traffic: 21
- MAD of time delta of user traffic: 95  

MAD = Median absolute deviation

## What does it mean?

Simply put:
- If beacon traffic ==> **uniform** distribution and **small** Median Absolute Deviation
- If user traffic ==> **skewed** distribution and **large** Median Absolute Deviation

## How can we detect beaconing traffic using this method?

# RITA to the rescue!

## RITA (Real Intelligence Threat Analytics)

![RITA Logo](./images/rita-logo.png)


Sponsored by [Active Countermeasures](https://activecountermeasures.com/).


RITA is an open source framework for network traffic analysis.

The framework ingests [Zeek Logs](https://www.zeek.org/) in TSV format, and currently supports the following major features:
 - **Beaconing Detection**: Search for signs of beaconing behavior in and out of your network
 - **DNS Tunneling Detection** Search for signs of DNS based covert channels
 - **Blacklist Checking**: Query blacklists to search for suspicious domains and hosts

## How RITA works

RITA analyzes time delta size distribution and data size(sent bytes) distribution of a traffic between the same source and the destination and calculates an overall score for time delta and data size.

## Time delta scoring
RITA calculates an overal time delta score from 3 different scores based on time delta. 

### 1. Time delta distribution
Perfect beacons should have symmetric delta time distribution. Bowley skewness measure is used to check symmetry. 

### 2. Median Absolute Deviation of time delta
Perfect beacons should have very low dispersion around the median of their delta times. Median Absolute Deviation is used to check dispersion.

### 3. Connection Count
If the total connection count of traffic is high, it's more likely a beacon and vice versa.

## Data size scoring
RITA calculates an overal data size score from 3 different scores based on data size(sent bytes is used for the analysis). 

### 1. Data size distribution
Perfect beacons should have symmetric data size distribution. Bowley skewness measure is used to check symmetry. 

### 2. Median Absolute Deviation of data size
Perfect beacons should have very low dispersion around the median of their data size. Median Absolute Deviation is used to check dispersion.

### 3. Data size smallness
If the size of sent bytes is small, it's more likely a beacon and vice versa.

---

# Implementing the algorithm using Jupyter notebook

We will:  
- Implement RITA beacon analyzer in Jupyter Notebook
  - Use dataset shared by Ali Alwashali ([@ali_alwashali](https://twitter.com/ali_alwashali))
    - Zeek logs from "malware-traffic-analysis.net" PCAP files, from 2013 to 2021
    - Suricata alerts triggered by the PCAP analysis
    
RITA beacon analyzer code: [https://github.com/activecm/rita/blob/master/pkg/beacon/analyzer.go](https://github.com/activecm/rita/blob/master/pkg/beacon/analyzer.go)

## Steps:  
0. Prepare data
1. Group connections between the same hosts and aggregate timestamps into a list
2. Calculate connection count and remove short sessions
3. Sort the connection timestamp ascending
4. Calculate time deltas
5. Generate variables required for score calculation (time delta and data size)
6. Calculate the score (time delta, data size, overall score)
7. Validation with Suricata alerts

In [80]:
import math
import pandas as pd
import numpy as np
import warnings
# Disable warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Preparing the Data

### Loading the data

In [81]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'id.orig_h'
f_dst_ip = 'id.resp_h'
f_dst_host = 'host'
f_dst_port = 'id.resp_p'
f_http_method = 'method'
f_sent_bytes = 'request_body_len'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method, f_sent_bytes]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
# columns to display after the analysis
columns_to_display = ['Score','tsScore','dsScore','conn_count',f_src_ip,f_dst_ip,f_dst_host,f_http_method, f_dst_port,f_sent_bytes,'deltas']

http_df = pd.read_csv('./sample-data/http-dataset.log', sep=f_delimiter)
http_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108967 entries, 0 to 108966
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ts                 108967 non-null  float64
 1   uid                108967 non-null  object 
 2   id.orig_h          108967 non-null  object 
 3   id.orig_p          108967 non-null  int64  
 4   id.resp_h          108967 non-null  object 
 5   id.resp_p          108967 non-null  int64  
 6   trans_depth        108967 non-null  int64  
 7   method             108967 non-null  object 
 8   host               108967 non-null  object 
 9   uri                108967 non-null  object 
 10  referrer           108967 non-null  object 
 11  version            108967 non-null  object 
 12  user_agent         108967 non-null  object 
 13  origin             108949 non-null  object 
 14  request_body_len   108967 non-null  int64  
 15  response_body_len  108967 non-null  int64  
 16  st

### Filtering Required Columns
```
df.loc[first_row_index:last_row_index , ['column1', 'column3']]
```

In [82]:
# get all rows and only the required colums
http_df = http_df.loc[:,columns_to_filter]
http_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108967 entries, 0 to 108966
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ts                108967 non-null  float64
 1   id.orig_h         108967 non-null  object 
 2   id.resp_h         108967 non-null  object 
 3   host              108967 non-null  object 
 4   id.resp_p         108967 non-null  int64  
 5   method            108967 non-null  object 
 6   request_body_len  108967 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 5.8+ MB


### Fixing the Timestamp Column Type

In [83]:
http_df[f_timestamp] = pd.to_datetime(http_df[f_timestamp], unit='s')
http_df.head(2)

,ts,id.orig_h,id.resp_h,host,id.resp_p,method,request_body_len
0,2013-06-19 00:25:23.332814848,192.168.122.178,173.247.253.210,www.insightcrime.org,80,GET,0
1,2013-06-19 00:25:23.961981184,192.168.122.178,93.171.172.220,93.171.172.220,80,GET,0


## Analysing the Data
Now the data is ready for statistical analysis. 

### Grouping the Connections
We can group the connections between the same hosts and aggregate the timestamps into a list. This way, we can use **pd.Series().diff** to easily calculate time deltas.  
First, let's group the traffic and aggregate.

In [84]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
http_df = http_df.groupby(columns_to_groupby).agg(list)
http_df.head(5)

ts  \
id.orig_h  id.resp_h       host                           id.resp_p method                                                      
1.8.31.101 104.168.98.206  104.168.98.206                 80        GET     [2019-08-31 13:07:54.941824, 2019-08-31 13:08:...   
           13.107.4.50     www.download.windowsupdate.com 80        GET                       [2019-08-31 13:00:38.149271040]   
           147.135.15.186  ip-api.com                     80        POST                      [2019-08-31 12:44:03.817665024]   
           170.238.117.187 170.238.117.187                8082      POST    [2019-08-31 13:07:53.975055872, 2019-08-31 13:...   
           172.217.12.228  www.google.com                 80        GET     [2019-08-31 12:44:28.299268864, 2019-08-31 12:...   

                                                                                           request_body_len  
id.orig_h  id.resp_h       host                           id.resp_p method                                   
1.8.31.101 104.168.98.206  104.168.98.206                 80        GET                           [0, 0, 0]  
           13.107.4.50     www.download.windowsupdate.com 80        GET                                 [0]  
           147.135.15.186  ip-api.com                     80        POST                               [25]  
           170.238.117.187 170.238.117.187                8082      POST    [268, 249, 348, 260, 3846, 152]  
           172.217.12.228  www.google.com                 80        GET                              [0, 0]

#### Reseting the index
We need to reset the index to use all the columns.

In [85]:
http_df.reset_index(inplace=True)
http_df.head(2)

,id.orig_h,id.resp_h,host,id.resp_p,method,ts,request_body_len
0,1.8.31.101,104.168.98.206,104.168.98.206,80,GET,"[2019-08-31 13:07:54.941824, 2019-08-31 13:08:...","[0, 0, 0]"
1,1.8.31.101,13.107.4.50,www.download.windowsupdate.com,80,GET,[2019-08-31 13:00:38.149271040],[0]


### Calculating Connection Count
Length of the timestamp list ('ts') = connection count (since we grouped the traffic and aggregated the timestamp into a list)  

In [86]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
http_df['conn_count'] = http_df[f_timestamp].apply(lambda x: len(x))
http_df.head(2)

,id.orig_h,id.resp_h,host,id.resp_p,method,ts,request_body_len,conn_count
0,1.8.31.101,104.168.98.206,104.168.98.206,80,GET,"[2019-08-31 13:07:54.941824, 2019-08-31 13:08:...","[0, 0, 0]",3
1,1.8.31.101,13.107.4.50,www.download.windowsupdate.com,80,GET,[2019-08-31 13:00:38.149271040],[0],1


### Remove short sessions
Filter out traffic where the connection count is quite small

In [87]:
http_df = http_df.loc[http_df['conn_count'] > 20]
http_df.shape

(433, 8)

#### Sorting the Timestamps
Apply a lambda function to each row on the specified columns.

In [88]:
http_df[f_timestamp] = http_df[f_timestamp].apply(lambda x: sorted(x))
http_df.head(2)

,id.orig_h,id.resp_h,host,id.resp_p,method,ts,request_body_len,conn_count
41,10.0.0.102,46.29.183.211,46.29.183.211,8080,POST,"[2019-10-25 16:30:30.652640, 2019-10-25 16:30:...","[477, 514, 434, 427, 471, 439, 435, 474, 447, ...",35
49,10.0.0.134,5.252.177.17,5.252.177.17,80,GET,"[2021-06-15 15:08:54.879534848, 2021-06-15 15:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",25


### Calculating Time Delta
Apply **pd.Series.diff()** to the 'ts' column, which is sorted, and assign the resulting list into a new column

In [89]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
http_df['deltas'] = http_df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
http_df.head(2)

,id.orig_h,id.resp_h,host,id.resp_p,method,ts,request_body_len,conn_count,deltas
41,10.0.0.102,46.29.183.211,46.29.183.211,8080,POST,"[2019-10-25 16:30:30.652640, 2019-10-25 16:30:...","[477, 514, 434, 427, 471, 439, 435, 474, 447, ...",35,"[2.0, 831.0, 880.0, 8.0, 862.0, 3.0, 120.0, 80..."
49,10.0.0.134,5.252.177.17,5.252.177.17,80,GET,"[2021-06-15 15:08:54.879534848, 2021-06-15 15:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",25,"[60.0, 60.0, 61.0, 60.0, 60.0, 60.0, 60.0, 60...."


### Generate variables
We need to generate the required variables to analyze time delta distribution, median absolute deviation, and connection count.

#### Variables for time delta dispersion

In [90]:
http_df['tsLow'] = http_df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
http_df['tsMid'] = http_df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
http_df['tsHigh'] = http_df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
http_df['tsBowleyNum'] = http_df['tsLow'] + http_df['tsHigh'] - 2*http_df['tsMid']
http_df['tsBowleyDen'] = http_df['tsHigh'] - http_df['tsLow']
http_df['tsSkew'] = http_df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
http_df['tsMadm'] = http_df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
http_df['tsConnDiv'] = http_df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 90)

#### Variables for data size dispersion

In [91]:
http_df['dsLow'] = http_df['request_body_len'].apply(lambda x: np.percentile(np.array(x), 20))
http_df['dsMid'] = http_df['request_body_len'].apply(lambda x: np.percentile(np.array(x), 50))
http_df['dsHigh'] = http_df['request_body_len'].apply(lambda x: np.percentile(np.array(x), 80))
http_df['dsBowleyNum'] = http_df['dsLow'] + http_df['dsHigh'] - 2*http_df['dsMid']
http_df['dsBowleyDen'] = http_df['dsHigh'] - http_df['dsLow']
http_df['dsSkew'] = http_df[['dsLow','dsMid','dsHigh','dsBowleyNum','dsBowleyDen']].apply(
    lambda x: x['dsBowleyNum'] / x['dsBowleyDen'] if x['dsBowleyDen'] != 0 and x['dsMid'] != x['dsLow'] and x['dsMid'] != x['dsHigh'] else 0.0, axis=1
    )
http_df['dsMadm'] = http_df['request_body_len'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))

In [92]:
http_df.head(3)

,id.orig_h,id.resp_h,host,id.resp_p,method,ts,request_body_len,conn_count,deltas,tsLow,...,tsSkew,tsMadm,tsConnDiv,dsLow,dsMid,dsHigh,dsBowleyNum,dsBowleyDen,dsSkew,dsMadm
41,10.0.0.102,46.29.183.211,46.29.183.211,8080,POST,"[2019-10-25 16:30:30.652640, 2019-10-25 16:30:...","[477, 514, 434, 427, 471, 439, 435, 474, 447, ...",35,"[2.0, 831.0, 880.0, 8.0, 862.0, 3.0, 120.0, 80...",2.6,...,-0.87349,74.5,195.333333,444.6,450.0,467.4,12.0,22.8,0.526316,5.0
49,10.0.0.134,5.252.177.17,5.252.177.17,80,GET,"[2021-06-15 15:08:54.879534848, 2021-06-15 15:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",25,"[60.0, 60.0, 61.0, 60.0, 60.0, 60.0, 60.0, 60....",60.0,...,0.00000,0.0,16.200000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
50,10.0.0.134,5.252.177.17,5.252.177.17,443,GET,"[2021-06-15 15:08:55.172674816, 2021-06-15 15:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1689,"[60.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...",0.0,...,0.00000,0.0,16.677778,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


### Calculating the score

In [93]:
# Time delta score calculation
http_df['tsSkewScore'] = 1.0 - abs(http_df['tsSkew'])
# If jitter is greater than 30 seconds, say 90 seconds, MadmScore might be zero
# It depends on how the jitter is implemented.
http_df['tsMadmScore'] = 1.0 - (http_df['tsMadm'] / 30.0)
http_df['tsMadmScore'] = http_df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
http_df['tsConnCountScore'] = (http_df['conn_count']) / http_df['tsConnDiv']
http_df['tsConnCountScore'] = http_df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
http_df['tsScore'] = (((http_df['tsSkewScore'] + http_df['tsMadmScore'] + http_df['tsConnCountScore']) / 3.0) * 1000) / 1000

# Data size score calculation of sent bytes
http_df['dsSkewScore'] = 1.0 - abs(http_df['dsSkew'])
# If data jitter is greater than 128 bytes, say 300 bytes, MadmScore might be zero
# Depends on how the jitter is implemented. 
http_df['dsMadmScore'] = 1.0 - (http_df['dsMadm'] / 128.0)
http_df['dsMadmScore'] = http_df['dsMadmScore'].apply(lambda x: 0 if x < 0 else x)
# Perfect beacons don't send to much data since they are idle and just checking in, 
# division by high number makes the score insensitive. 
# Making the smallness score more sensitive as it makes more sense. 
http_df['dsSmallnessScore'] = 1.0 - (http_df['dsMid'] / 8192.0)
http_df['dsSmallnessScore'] = http_df['dsSmallnessScore'].apply(lambda x: 0 if x < 0 else x)
http_df['dsScore'] = (((http_df['dsSkewScore'] + http_df['dsMadmScore'] + http_df['dsSmallnessScore']) / 3.0) * 1000) / 1000

# Overal Score calculation
http_df['Score'] = (http_df['dsScore'] + http_df['tsScore']) / 2

http_df.sort_values(by= 'Score', ascending=False, inplace=True, ignore_index=True)
http_df[columns_to_display].head(30)


,Score,tsScore,dsScore,conn_count,id.orig_h,id.resp_h,host,method,id.resp_p,request_body_len,deltas
0,1.0,1.0,1.0,73,192.168.137.85,202.152.48.35,www.twentyone-development.com,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1.0,1.0,1.0,84,10.8.26.101,166.62.111.64,mysocalledchaos.com,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1.0,1.0,1.0,43,10.9.26.101,50.19.218.16,api.ipify.org,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1.0,1.0,1.0,25,10.9.26.101,23.23.83.153,api.ipify.org,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1.0,1.0,1.0,77,192.168.200.8,185.141.24.71,webintercom76delivery.net,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[60.0, 60.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0..."
5,1.0,1.0,1.0,21,192.168.204.130,194.110.243.156,www.cairnsmhor.com,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,1.0,1.0,1.0,51,10.9.13.101,81.92.202.176,tain0077.warzonedns.com,POST,5200,"[0, 0, 0, 0, 0, 0, 17, 0, 36, 538, 0, 0, 0, 0,...","[6.0, 0.0, 0.0, 66.0, 9.0, 0.0, 3.0, 0.0, 24.0..."
7,1.0,1.0,1.0,25,10.9.12.104,108.61.99.79,108.61.99.79,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1.0,1.0,1.0,68,192.168.204.134,185.31.209.83,diq.endpointcash.asia,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
9,1.0,1.0,1.0,35,192.168.204.134,63.141.137.25,www.perfumelover.co.uk,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## Display the traffic with high scores

In [94]:
http_df.loc[http_df['Score'] > 0.80, columns_to_display]

,Score,tsScore,dsScore,conn_count,id.orig_h,id.resp_h,host,method,id.resp_p,request_body_len,deltas
0,1.000000,1.000000,1.000000,73,192.168.137.85,202.152.48.35,www.twentyone-development.com,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1.000000,1.000000,1.000000,84,10.8.26.101,166.62.111.64,mysocalledchaos.com,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1.000000,1.000000,1.000000,43,10.9.26.101,50.19.218.16,api.ipify.org,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1.000000,1.000000,1.000000,25,10.9.26.101,23.23.83.153,api.ipify.org,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1.000000,1.000000,1.000000,77,192.168.200.8,185.141.24.71,webintercom76delivery.net,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[60.0, 60.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0..."
...,...,...,...,...,...,...,...,...,...,...,...
400,0.821720,0.643441,1.000000,27,10.6.3.102,176.119.156.128,a-zcorner.com,GET,80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 64.0, 170.0, 169.0, 28.0, 0.0, 140.0, 16..."
401,0.814274,1.000000,0.628547,27,10.10.27.101,195.123.209.74,miligratonylaonella.com,POST,80,"[944, 992, 1008, 1024, 1040, 1056, 1072, 1072,...","[3.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, ..."
402,0.813192,0.642049,0.984334,24,10.8.12.101,195.22.26.248,mtcunlocker.info,POST,80,"[196, 192, 198, 192, 196, 194, 190, 192, 194, ...","[73.0, 61.0, 61.0, 71.0, 73.0, 60.0, 83.0, 809..."
403,0.807864,0.714035,0.901693,21,10.8.29.102,119.28.15.177,amellet.bit,POST,80,"[848, 240, 832, 240, 336, 368, 368, 368, 368, ...","[47.0, 15.0, 52.0, 49.0, 3.0, 4.0, 2.0, 2.0, 3..."


# Validation with Suricata Alerts

In [95]:
suricata_df = pd.read_csv('./sample-data/suricata alerts.csv', sep=',')
suricata_df.loc[suricata_df['dest_ip'].isin(['107.181.187.14','5.199.162.3','23.74.28.9','35.198.166.240',
                                             '193.33.134.7','31.184.192.202','5.149.222.125','185.180.198.24',
                                             '173.254.231.111','139.60.161.74','192.254.79.71','185.68.93.18',
                                             '69.16.143.110','37.59.68.215','31.44.184.33','178.74.214.194']),
                                             ['src_ip','dest_ip','alert.signature']].drop_duplicates()

,src_ip,dest_ip,alert.signature
1392,10.17.6.93,139.60.161.74,ET MALWARE Cobalt Strike Beacon Observed
1772,10.17.6.93,139.60.161.74,ET HUNTING GENERIC SUSPICIOUS POST to Dotted Q...
8151,10.5.26.4,5.199.162.3,ET MALWARE Cobalt Strike Malleable C2 Profile ...
11403,10.2.2.101,192.254.79.71,ET MALWARE Cobalt Strike Beacon Observed
11433,10.2.2.101,192.254.79.71,ET JA3 Hash - [Abuse.ch] Possible Dridex
13811,10.2.2.101,192.254.79.71,ET HUNTING GENERIC SUSPICIOUS POST to Dotted Q...
20123,10.2.2.101,192.254.79.71,ET ADWARE_PUP Fun Web Products Spyware User-Ag...
62016,10.7.25.101,31.44.184.33,ET MALWARE Cobalt Strike Beacon Observed
72576,10.7.25.101,31.44.184.33,ET HUNTING GENERIC SUSPICIOUS POST to Dotted Q...
73327,10.7.22.101,31.44.184.33,ET HUNTING GENERIC SUSPICIOUS POST to Dotted Q...
